In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob

In [38]:
plt.imread('train/F0002/MID1/P00012_face2.jpg').shape

(224, 224, 3)

In [39]:
from keras.models import Model
from keras.layers import Convolution2D, Flatten
from keras.layers import Dense, Input, Dropout, concatenate

In [129]:
inp1 = Input(shape = (224,224,3))
x = Convolution2D(filters=32, kernel_size=(3, 3), strides=(2, 2), activation='relu')(inp1)
x = Convolution2D(filters=64, kernel_size=(3, 3), strides=(2, 2), activation='relu')(x)
x = Convolution2D(filters=128, kernel_size=(3, 3), strides=(2, 2), activation='relu')(x)
x = Convolution2D(filters=256, kernel_size=(5, 5), strides=(2, 2), activation='relu')(x)
x = Convolution2D(filters=512, kernel_size=(5, 5), strides=(2, 2), activation='relu')(x)
x = Flatten()(x)
x = Dropout(rate=0.5)(x)
x1 = Dense(64,activation='relu')(x)

inp2 = Input(shape = (224,224,3))
x = Convolution2D(filters=32, kernel_size=(3, 3), strides=(2, 2), activation='relu')(inp2)
x = Convolution2D(filters=64, kernel_size=(3, 3), strides=(2, 2), activation='relu')(x)
x = Convolution2D(filters=128, kernel_size=(3, 3), strides=(2, 2), activation='relu')(x)
x = Convolution2D(filters=256, kernel_size=(5, 5), strides=(2, 2), activation='relu')(x)
x = Convolution2D(filters=512, kernel_size=(5, 5), strides=(2, 2), activation='relu')(x)
x = Flatten()(x)
x = Dropout(rate=0.5)(x)
x2 = Dense(64,activation='relu')(x)

x = concatenate([x1,x2])
output = (Dense(1,activation='sigmoid'))(x)

model = Model(inputs=[inp1,inp2],outputs=output)
model.compile(optimizer='adam',loss='binary_crossentropy')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_35 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
input_36 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_167 (Conv2D)             (None, 111, 111, 32) 896         input_35[0][0]                   
__________________________________________________________________________________________________
conv2d_172 (Conv2D)             (None, 111, 111, 32) 896         input_36[0][0]                   
__________________________________________________________________________________________________
conv2d_168

In [130]:
def prepare_pair(str1,str2):
    # return the images provided current string path as arrays
    a = plt.imread(str1)
    b = plt.imread(str2)
    return (a,b)

def make_pairs(str1,str2):
    # prepare a list of all image pairs for two folders
    l1 = glob.glob('train/'+str1+'/*')
    l2 = glob.glob('train/'+str2+'/*')
    if (len(l1)>len(l2)):
        l1,l2 = l2,l1
    output = []
    for i in range(len(l1)):
        for j in range(i,len(l2)):
            output.append((l1[i],l2[j]))
    return output

def generate_batch(i,j=None):
    # make a batch ready for training
    # i for a single entry from the pairs dataframe
    # i,j for two individuals that are not related
    if (j is None):
        related = True
        pairs = make_pairs(*list(relationships.iloc[i]))
    else:
        related = False
        pairs = make_pairs(i,j)
    size = len(pairs)
    X1 = np.zeros((size,224,224,3))
    X2 = np.zeros((size,224,224,3))
    if (related):
        y = np.ones((size),dtype=int)
    else:
        y = np.zeros((size),dtype=int)
    i = 0
    for pair in pairs:
        X1[i],X2[i] = prepare_pair(*pair)
        i+=1
    X1/=255.
    X2/=255.
    return X1, X2, y

In [131]:
# related
a,b,c = generate_batch(222)
a.shape,b.shape,c.shape

((7, 224, 224, 3), (7, 224, 224, 3), (7,))

In [132]:
# not related
a,b,c = generate_batch(relationships.iloc[3][0],relationships.iloc[123][1])
a.shape,b.shape,c.shape

((18, 224, 224, 3), (18, 224, 224, 3), (18,))

In [ ]:
# training
for epoch in range(100):
    print('Epoch:',epoch)
    # one batch on related
    i = np.random.randint(len(relationships))
    X1, X2, y = generate_batch(i)
    print('\trelated',len(X1))
    model.train_on_batch([X1,X2],y)
    # one batch on unrelated
    i = np.random.randint(len(relationships))
    j = i
    while (j==i):
        j = np.random.randint(len(relationships))
    a = relationships.iloc[i][np.random.randint(2)]
    b = relationships.iloc[j][np.random.randint(2)]
    X1, X2, y = generate_batch(a,b)
    print('\tunrelated',len(X2))
    model.train_on_batch([X1,X2],y)

Epoch: 0
	related 6
	unrelated 5
Epoch: 1
	related 3
	unrelated 9
Epoch: 2
	related 0
	unrelated 15
Epoch: 3
	related 36
	unrelated 7
Epoch: 4
	related 17
	unrelated 119
Epoch: 5
	related 3
	unrelated 5
Epoch: 6
	related 30
	unrelated 56
Epoch: 7
	related 2
	unrelated 13
Epoch: 8
	related 4
	unrelated 1
Epoch: 9
	related 7
	unrelated 6
Epoch: 10
	related 10
	unrelated 27
Epoch: 11
	related 5
	unrelated 6
Epoch: 12
	related 15
	unrelated 7
Epoch: 13
	related 7
	unrelated 35
Epoch: 14
	related 1
	unrelated 7
Epoch: 15
	related 8
	unrelated 126
Epoch: 16
	related 4
	unrelated 19
Epoch: 17
	related 33
	unrelated 54
Epoch: 18
	related 35
	unrelated 39
Epoch: 19
	related 10
	unrelated 8
Epoch: 20
	related 51
	unrelated 45
Epoch: 21
	related 30
	unrelated 0
Epoch: 22
	related 0
	unrelated 261
Epoch: 23
	related 27
	unrelated 68
Epoch: 24
	related 0
	unrelated 17
Epoch: 25
	related 10
	unrelated 5
Epoch: 26
	related 27
	unrelated 21
Epoch: 27
	related 0
	unrelated 2
Epoch: 28
	related 2
	unrel

In [138]:
# evaluation
for epoch in range(10):
    print('Epoch:',epoch)
    # one batch on related
    i = np.random.randint(len(relationships))
    X1, X2, y = generate_batch(i)
    if (len(y)>0):
        p = model.predict([X1,X2])
        p[p>0.5]=1
        p[p<=0.5]=0
        print('\trelated',len(X1),(y==p).mean())
    # one batch on unrelated
    i = np.random.randint(len(relationships))
    j = i
    while (j==i):
        j = np.random.randint(len(relationships))
    a = relationships.iloc[i][np.random.randint(2)]
    b = relationships.iloc[j][np.random.randint(2)]
    X1, X2, y = generate_batch(a,b)
    if (len(y)>0):
        p = model.predict([X1,X2])
        p[p>0.5]=1
        p[p<=0.5]=0
        print('\tunrelated',len(X1),(y==p).mean())

Epoch: 0
	related 9 1.0
	unrelated 4 0.0
Epoch: 1
	related 268 1.0
	unrelated 34 0.08823529411764706
Epoch: 2
	related 21 1.0
	unrelated 5 0.0
Epoch: 3
	related 24 1.0
	unrelated 1 0.0
Epoch: 4
	related 3 1.0
	unrelated 45 0.0
Epoch: 5
	unrelated 6 0.0
Epoch: 6
	related 14 1.0
	unrelated 6 0.0
Epoch: 7
	related 55 1.0
	unrelated 175 0.017142857142857144
Epoch: 8
	related 4 1.0
	unrelated 12 0.0
Epoch: 9
	related 1 1.0
	unrelated 9 0.0
